#### Function To Get Ticker Data

In [ ]:
# Pull in historical data from Yahoo Finance for a specified ticker
import yfinance as yf
import pandas as pd
from pathlib import Path
import datetime
import os

def get_ticker_data(ticker, period, interval):
    '''This function pulls historical data for a specified ticker from Yahoo Finance using yfinance.
    If you do not have yfinance installed, run pip install yfinance
    If the current trading day has not closed it will be removed from the data.
    
    Inputs:
        ticker = ticker symbol for the trade instrument. (str)
        period = data period to download 
            available options: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
        interval = data interval.
            available options 1d, 5d, 1wk, 1mo, 3mo     
    '''
    # set folder and file variables
    folder_name = 'Data'
    file_name = ticker
    file = os.path.join(folder_name, file_name + '.csv')
    p = Path(folder_name)
    
    # Get data amd drop the current trading day
    data = yf.Ticker(ticker)
    data_df = data.history(period=period,
                                interval=interval,
                                actions=False,
                                back_adjust=True).reset_index()
    data_df = data_df.iloc[:-1]
    
    # Check if folder and file exist
    if os.path.exists(file):
        
    # Read in historical data CSV, convert dtype of Date column, append DFs, drop dups, and overwrite csv  
        historical_data = pd.read_csv(file)
        historical_data['Date'] = pd.to_datetime(historical_data.Date)
        historical_data = (historical_data.append(data_df)
                  .reset_index()
                  .drop(columns='index')
                 )
        data_df = historical_data.drop_duplicates(keep='last')
        data_df.to_csv(Path(file), index=False) 
        strSuccess = f'Appended data to {file}.'
        
    # Check if directory exist and save df to Data folder as a csv using the ticker as the name
    elif os.path.isdir(folder_name):
        data_df.to_csv(Path(p, ticker + '.csv'), index=False)
        strSuccess = f'Wrote your file to the {p} folder as {file_name}.csv.'
        
    # make the Data folder and save df to csv using the ticker as the name
    else:
        os.makedirs(folder_name)
        data_df.to_csv(Path(p, ticker + '.csv'), index=False)
        strSuccess = f'A folder named, {folder_name}, was created and you file was save in it as {file_name}.csv.'
    return(strSuccess, data_df.tail())

#### Clean Ticker Data

In [ ]:
# file paths
cl_path = Path('Data/CL=F.csv')
uup_path = Path('Data/UUP.csv')

# Read files to a dataframe
cl_df = pd.read_csv(cl_path, index_col='Date', parse_dates=True, infer_datetime_format=True).sort_index()
uup_df = pd.read_csv(uup_path, index_col='Date', parse_dates=True, infer_datetime_format=True).sort_index()

# Drop columns to have just closing price
cl_close = cl_df.drop(columns=['Open', 'High', 'Low', 'Volume'])
uup_close = uup_df.drop(columns=['Open', 'High', 'Low', 'Volume'])

# Change column name to ticker name
cl_close.columns = ['CL']
uup_close.columns = ['UUP']

# # Combine the two dataframes
combined_cl_uup = pd.concat([cl_close, uup_close], axis='columns', join='inner')